[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/M-Talha-Farooqi/Machine-Learning-CourseWork/blob/main/Assignments/Assignment-3/Random_Forest_Classifier.ipynb)

## Random Forest Classifier Explanation

This code implements a Random Forest Classifier from scratch using a custom Decision Tree Classifier.

### Node Class

The `Node` class represents a node in the decision tree. Each node can be either an internal node or a leaf node.

- `feature_index`: The index of the feature used for splitting at this node (for internal nodes).
- `threshold`: The threshold value for splitting at this node (for internal nodes).
- `left`: The left child node.
- `right`: The right child node.
- `info_gain`: The information gain achieved by splitting at this node.
- `value`: The predicted class value for a leaf node.

### DecisionTreeClassifier Class

The `DecisionTreeClassifier` class implements a single decision tree.

- `__init__(self, min_samples_split=2, max_depth=2)`: Initializes the tree with minimum samples required to split a node and maximum depth of the tree.
- `fit(self, X, Y)`: Trains the decision tree on the input data `X` and target labels `Y`. It calls the `build_tree` method to recursively build the tree.
- `build_tree(self, dataset, curr_depth=0)`: Recursively builds the decision tree. It finds the best split at each node based on information gain and creates child nodes.
- `calculate_leaf_value(self, Y)`: Calculates the most frequent class in the target labels `Y` for a leaf node.
- `get_best_split(self, dataset, num_samples, num_features)`: Finds the best feature and threshold to split the data based on the maximum information gain.
- `split(self, dataset, feature_index, threshold)`: Splits the dataset into two subsets based on the specified feature and threshold.
- `information_gain(self, parent, l_child, r_child, mode="gini")`: Calculates the information gain using either Gini impurity or entropy.
- `entropy(self, y)`: Calculates the entropy of the target labels `y`.
- `gini_index(self, y)`: Calculates the Gini impurity of the target labels `y`.
- `predict(self, X)`: Predicts the class labels for the input data `X`.
- `make_prediction(self, x, tree)`: Traverses the decision tree to predict the class label for a single data point `x`.

### RandomForestClassifier Class

The `RandomForestClassifier` class implements the random forest algorithm using multiple Decision Tree Classifiers.

- `__init__(self, n_estimators=5, min_samples_split=2, max_depth=5)`: Initializes the random forest with the number of decision trees (`n_estimators`), minimum samples per split, and maximum depth for each tree.
- `fit(self, X, Y)`: Trains the random forest. It creates `n_estimators` decision trees, trains each tree on a bootstrapped sample of the data, and stores them in the `trees` list.
- `predict(self, X)`: Predicts the class labels for the input data `X`. It gets predictions from each decision tree and uses a majority vote to determine the final prediction.

# Importing libraries

In [15]:
import numpy as np
from collections import Counter
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Random Forest Classifier Model

In [16]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

# ------------------------------
# Decision Tree Classifier
# ------------------------------
class DecisionTreeClassifier:
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def fit(self, X, Y):
        if Y.ndim == 1:
            Y = Y.reshape(-1, 1)
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)

        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split and "info_gain" in best_split and best_split["info_gain"] > 0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)


    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)

    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")

        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)

            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)

                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    current_info_gain = self.information_gain(y, left_y, right_y, "gini")

                    if current_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = current_info_gain
                        max_info_gain = current_info_gain

        return best_split

    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right

    def information_gain(self, parent, l_child, r_child, mode="gini"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode == "gini":
            gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
        return gain

    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy

    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls ** 2
        return 1 - gini

    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions

    def make_prediction(self, x, tree):
        if tree.value is not None:
            return tree.value

        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

# ------------------------------
# Random Forest Classifier
# ------------------------------
class RandomForestClassifier:
    def __init__(self, n_estimators=5, min_samples_split=2, max_depth=5):
        self.n_estimators = n_estimators
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, Y):
        if Y.ndim == 1:
            Y = Y.reshape(-1, 1)
        self.trees = []
        for _ in range(self.n_estimators):
            idxs = np.random.choice(len(X), len(X), replace=True)
            X_sample, Y_sample = X[idxs], Y[idxs]
            tree = DecisionTreeClassifier(min_samples_split=self.min_samples_split, max_depth=self.max_depth)
            tree.fit(X_sample, Y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [Counter(row).most_common(1)[0][0] for row in tree_preds]
        return y_pred

# Loading DataSet

In [17]:
# ------------------------------
# Test on Iris Dataset
# ------------------------------
# Load dataset
data = load_iris()
X = data.data
Y = data.target

# Train - Test Split

In [18]:
# Split into train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Model Training

In [19]:
# Train random forest
clf = RandomForestClassifier(n_estimators=5, max_depth=5)
clf.fit(X_train, Y_train)

# Predict from Model

In [20]:
# Make predictions
preds = clf.predict(X_test)

# Model Evaluation

In [21]:
# Evaluate accuracy
acc = accuracy_score(Y_test, preds)
print("Random Forest Accuracy:", acc*100)

Random Forest Accuracy: 100.0
